## Preparing the Workplace

### Libraries

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import re

### Extra Settings

In [ ]:
# Setting the default size of visualisations.
sns.set(rc={'figure.figsize': (9, 6)})

# Ignoring red warnings
pd.options.mode.chained_assignment = None

### Functions

In [ ]:
# ========== Here's the main function for the basic data overview.

def do_basic_overview(data):
    print()
    print(12*'=', 'Basic Info', 12*'=')
    data.info()
    print('\n\n')
    print(12*'=', 'First Five Rows', 12*'=')
    display(data.head())
    print('\n\n')
    print(12*'=', 'Last Five Rows', 12*'=')
    display(data.tail())
    print('\n\n')
    print(12*'=', 'Duplicates', 12*'=')
    print()
    if data.duplicated().sum() > 0:
        print(f'Amount of duplicated rows: {data.duplicated().sum()}.')
        data.drop_duplicates(inplace=True, ignore_index=True)
        print('Duplicated rows were dropped.')
    else:
        print('None.')
        
        
# Creating histograms for all columns with numeric values in search for some obvious anomalies.
def numeric_cols_hist(data, width, height):
        cols = data.select_dtypes(include=('number')).columns.to_list()
        data[cols].hist(bins=30, figsize=(width, height))
        plt.show()
        print('Here, the density of values in each numeric column is shown.')

In [ ]:
# Makes a dataframe to overview datatypes
# for each column of the original dataframe

def make_df_dtype(data):
    cols, dtype_col, specimens, nunique, null_share = [], [], [], [], []
    for column in data:
        cols.append(column)
        
        this_dtype = data[column].dtype
        dtype_col.append(this_dtype)
        
        specimen = data.loc[data[column].first_valid_index(), column]
        specimens.append(specimen)
        
        nunique.append(data[column].nunique())
        
        null_sum = data[column].isna().sum()
        null_to_len = null_sum / len(data[column])
        null_share.append(f'{null_to_len:.2%}')
        
    df = pd.DataFrame(list(zip(dtype_col,
                               specimens,
                               nunique,
                              null_share)),
                      index=cols)    
    df.columns=['dtype',
                'specimen',
                'nunique',
               'null_share']
 
    return (df.style.set_properties(**{'text-align': 'left'}))

In [ ]:
# Calculating share of null values for the column.
def print_null_share(column):
    null_sum = data[column].isna().sum()
    null_to_len = null_sum / len(data[column])
    print(f"'{column}', percentage of null-values: {null_to_len:.2%}.")

### Data

In [ ]:
data = pd.read_csv('/kaggle/input/revealing-insights-from-youtube-video-and-channe/YouTubeDataset_withChannelElapsed.csv')

## Data Overview and Preprocessing 

This dataset contains YouTube video and channel metadata.

Let us take a first glance on the information we have.

### First Glance

NB: `elapsed time` - average time spent watching a video per user.

|Column name|Description|
|:----|:----|
|totalviews/channelelapsedtime|Ratio of total views to channel elapsed time. (Ratio)|
|channelViewCount|Total number of views for the channel. (Integer)|
|likes/subscriber|Ratio of likes to subscribers. (Ratio)|
|views/subscribers|Ratio of views to subscribers. (Ratio)|
|subscriberCount|Total number of subscribers for the channel. (Integer)|
|dislikes/views|Ratio of dislikes to views. (Ratio)|
|comments/subscriber|Ratio of comments to subscribers. (Ratio)|
|channelCommentCount|Total number of comments for the channel. (Integer)|
|likes/dislikes|Ratio of likes to dislikes. (Ratio)|
|comments/views|Ratio of comments to views. (Ratio)|
|dislikes/subscriber|Ratio of dislikes to subscribers. (Ratio)|
|totviews/totsubs|Ratio of total views to total subscribers. (Ratio)|
|views/elapsedtime|Ratio of views to elapsed time. (Ratio)|


In [ ]:
make_df_dtype(data)

In [ ]:
data.describe().T

### First insights

- We have mostly numeric data.
- In some cases, there are numericish-looking timestamps and categorical values (for instance, video categories: we will decipher them later by borrowing their meaning from [here](https://gist.github.com/dgp/1b24bf2961521bd75d6c)).
- For some reasons, some ratio columns ('likes/subscriber' etc.) have negative values in it: is `-1` a placeholder for null-values? The column dedicated to ratio of likes to dislikes predominantly consists of `-2`'s. We need to do something about it.
- Not all the columns are covered in depth in the data description. For instance, we don't know what unit is used in case of 'elapsed_time' (seconds? minutes?).

As our next step, we will do some housekeeping and go through a deeper data overview.

### Further Overview and Housekeeping

In [ ]:
data.info()

First, let's do some preparatory work with columns names and data types.

In [ ]:
# Converting column names to snake_case.
pattern = re.compile(r'(?<!^)(?=[A-Z])')

for column in data.columns:
    column_upd = pattern.sub('_', column).lower()
    data = data.rename(columns={column: column_upd})
    
data = data.rename(columns={
    'totalviews/channelelapsedtime': 'total_views/channel_elapsed_time',
                    'channelelapsedtime': 'channel_elapsed_time',
                    'totvideos/videocount': 'total_videos/video_count',
                    'elapsedtime': 'elapsed_time',
                    'totviews/totsubs': 'total_views/total_subscribers',
                    'views/elapsedtime': 'views/elapsed_time'})


# Let's drop the index column.
data = data.drop(['index'], axis=1)


# Let's identify our dates as dates.
data['video_published'] = pd.to_datetime(data['video_published'])


# For now, let us cast the ids of video categories as strings they practically are.
data['video_category_id'] = data['video_category_id'].astype('str')

To make our futher visualisations more clear, let us decipher video categories.

In [ ]:
data['video_category'] = np.nan

for item in [['2', 'Autos & Vehicles'], ['1', 'Film & Animation'],
             ['10', 'Music'], ['15', 'Pets & Animals'],
             ['17', 'Sports'], ['18', 'Short Movies'],
             ['19', 'Travel & Events'], ['20', 'Gaming'],
             ['21', 'Videoblogging'], ['22', 'People & Blogs'],
             ['23', 'Comedy'], ['24', 'Entertainment'],
             ['25', 'News & Politics'], ['26', 'Howto & Style'],
             ['27', 'Education'], ['28', 'Science & Technology'],
             ['29', 'Nonprofits & Activism'], ['30', 'Movies'],
             ['31', 'Anime/Animation'], ['32', 'Action/Adventure'],
             ['33', 'Classics'], ['34', 'Comedy'], ['35', 'Documentary'],
             ['36', 'Drama'], ['37', 'Family'], ['38', 'Foreign'],
             ['39', 'Horror'], ['40', 'Sci-Fi/Fantasy'], ['41', 'Thriller'],
             ['42', 'Shorts'], ['43', 'Shows'], ['44', 'Trailers']]:
    
    (data
     .loc[data['video_category_id'] == item[0],
          'video_category']) = item[1]

print_null_share('video_category')

In [ ]:
make_df_dtype(data)

In [ ]:
do_basic_overview(data)

Apparently, we had some explicitly duplicated rows.
<br>About 3.3% of our original data, to put it relatively (this sounds less frightening than 19&nbsp;186 rows).

We will be on our toes to catch implicit ones, too, if needed.

### Density of Numeric Values

In [ ]:
numeric_cols_hist(data, width=25, height=19)

- As we can see, most of our distributions have a very strong right skewness.
<br>This means, there is a long 'tail' to the right. Judging by the ticks on the x-axis, sometimes it reaches quite impressive values, but it can't be seen at the current scale.
<br>At the same time, in most cases, there is a peak on the left, somewhere closer to the zero point.
- We still need to do something about mysterious negative ratios.

### Dealing with Negative Values (where they don't belong)

It is impossible for a ratio to be negative.

A number of something can't be less than zero as well.

We can asume that some negative value can stand for the gap in the original data being, in fact, a placeholder.<br>However, in this case, we should see some pattern and uniformity of usage. For instance, only `-1` shall be used.

In other cases, we need to do something about it.
<br><br>
Let us explore the columns 'below zero'.<br>After this, we will wipe all the negative values away.<br>It will be quite easy to fill the null values once again or drop them completely, if needed.

In [ ]:
# Let's keep columns with negative values in a list.
columns_below_zero = []

# We will go through the columns with numeric values of our dataset.
for column_name in data.select_dtypes(include='number').columns.to_list():
    if data[column_name].min() < 0:
        columns_below_zero.append(f'{column_name}')
        print(f"'{column_name}'")
        
        # We will print unique negative values.
        # At the same time, we need to set a limit to keep it quick.
        print("Unique values 'below zero':")
        i = 0
        for elem in data.loc[data[column_name] < 0][column_name].unique():
            print(f'{elem}')
            i += 1
            # If more than 5 values is already printed, it stops.
            if i > 4:
                print('- and others.')
                break
                
        print(f"Minimum value: {data[column_name].min()}")
        
        # Now, we will wipe the negative values away.
        print_null_share(column_name)
        print('--> Now, we are erasing negative values.')
        data.loc[data[column_name] < 0, column_name] = np.nan
        print_null_share(column_name)
        print()

In [ ]:
make_df_dtype(data[columns_below_zero])

Having in mind the uniformity of usage of `-1` in several columns, we could have thought about being a substitute for null-values.

However, in other columns the situation is more nuanced, and there is a bigger variety of possible negative values that can't be explained by conventions and/or some natural causes.

**We will fill the gaps with median for this or that video category**<br>
In most cases, most of the values are positive: negative values are making up way less than 5-6% of this or that particular column. Whatever we do, hopefully, we will not affect the majority of rows and overall pattern.<br>To make the values less artificial and not to erase rows, we will fill the null-values with median value calculated for each video category.

**We will drop 'likes/dislikes' column**<br>
There is only one exception, and it is 'likes/dislikes' column: it is almost completely 'below zero'! There is no need to keep it in our dataset.

In [ ]:
# Filling the gaps.
for column_name in columns_below_zero:
    data[column_name] = data[column_name].fillna(data
                                               .groupby('video_category')
                                               [column_name].transform("median"))

In [ ]:
# Dropping the column.
data = data.drop(['likes/dislikes'], axis=1)

In [ ]:
make_df_dtype(data)

In [ ]:
data.info()

### Organising the Features

Before we dive into our futher exploration, let's bring some order into our list of our numeric non-categorical features.

They can be naturally divided into three categories:
- dealing with videos (even though 'video_category_id' and 'video_published_year' are categorical, we will classify list it here as well)
- dealing with channels
- dealing with ratios

Firstly, we will divide the list of columns into three categories based on their names.

We will be able to make some manual edits after that (if needed).

In [ ]:
video_details = []
channel_details = []
ratios = []
for col in data.select_dtypes(include='number').columns.to_list():
    if "/" not in col:
        if 'video' in col:
            video_details.append(col)
        if 'channel' in col:
            channel_details.append(col)
    elif '/' in col:
        ratios.append(col)
    else:
        print(f'!!! Was not classified: {col}.\n')

Let's keep our orginised features in a special `data_dict` where they can be easily accessible.

In [ ]:
data_dict = {
    'video_details': video_details,
    'channel_details': channel_details,
    'ratios': ratios
}

Let's check the result.

In [ ]:
data_dict['video_details']

In [ ]:
data_dict['channel_details']

In [ ]:
data_dict['ratios']

### Let's Built Some Boxplots to Take a Look at the Numeric Values

In [ ]:
for item in data_dict['video_details']:
    sns.boxplot(x=data[item])
    plt.title(f"Information about videos:\n'{item}'", fontsize=25)
    plt.show()
    print()

In [ ]:
for item in data_dict['channel_details']:
    sns.boxplot(x=data[item])
    plt.title(f"Information about channels:\n'{item}'", fontsize=25)
    plt.show()
    print()

As our histograms have already shown us, we have a very strong right skewness.

The *box* (the lowest and highest data points in the data set excluding any statistically defined outliers) is indistinguishable in most of the cases, leaving us with plenty of outliers (diamond-shaped black points; sometimes they are merging together).

In [ ]:
# we can cast the ids of video categories back to integers and add it to the list of categories
data.video_category_id = data.video_category_id.astype('int')

data_dict['video_details'].append('video_category_id')
data_dict['video_details']

### Time range

In [ ]:
data['video_published'].hist()
plt.title("'video_published' values", fontsize=25)
plt.show()

Let's add a new column for years: it can be useful for dynamic analysis.

In [ ]:
data['video_published_year'] = data['video_published'].dt.year
data[['video_published_year', 'video_published']].head()

Apparently, our histogram offered us a shortened version of list of years.

Let's group our data by years.

In [ ]:
(
    data.groupby('video_published_year')
    .agg(total_videos_published=('video_id', 'nunique'),
         total_channels_with_new_videos=('channel_id', 'nunique'))
).style.background_gradient(cmap='Blues_r')

### Linear Correlations

In [ ]:
correlations = round(data.corr(), 2)

In [ ]:
print('Significant negative linear correlation:')
for column_name in correlations.columns.to_list():
    inds = (correlations
            .loc[correlations[column_name] < -0.5]
            .index.to_list())       
    for elem in inds:
        print(f"- `{elem}` - `{column_name}`")

In [ ]:
print('Very strong positive linear correlation:')
for column_name in correlations.columns.to_list():
    inds = (correlations.loc[correlations[column_name] >= 0.9].index.to_list())
    for elem in inds:
        if elem != column_name:
            print(f"- `{elem}` - `{column_name}`")

In [ ]:
print('Strong positive linear correlation:')
for column_name in correlations.columns.to_list():
    inds = (correlations
            .loc[(correlations[column_name] <= 0.9) & (correlations[column_name] >= 0.7)]
            .index.to_list())       
    for elem in inds:
        print(f"- `{elem}` - `{column_name}`")

In [ ]:
sns.heatmap(data.corr(), cmap='viridis')
plt.show()

**Negative correlations**<br>
Negative linear correlation means the growth of something goes hand-by-hand with something else's decrease.<br>In case of our data, those correlations are time-related and thus are natural.
<br><br>

As we were able to see, there are some significant positive linear correlations.

**Very strong positive correlation**
- `channel_view_count` - `total_views/channel_elapsed_time`
- `views/elapsed_time` - `video_view_count`
<br><br>

**Strong positive linear correlation**
- `subscriber_count` - `total_views/channel_elapsed_time`
- `subscriber_count` - `channel_view_count`
- `video_like_count` - `video_view_count`
- `video_dislike_count` - `video_view_count`
- `video_comment_count` - `video_like_count`
- `views/elapsed_time` - `video_dislike_count`


### Organising the Categories

In [ ]:
categories = (data
              .select_dtypes(include='object')
              .columns.to_list())
categories.append('video_published_year')

print('We have the following categorical columns:')
for column_name in categories:
    nunique = data[column_name].nunique()
    print(f'- {column_name}: {nunique} unique values')

- We have a decent amount of video and channel ids: about half a million and a bit less/more in each case. It can be a blessing and a curse, if not handled wisely.
- 18 thematic categories for videos is not that much. Yet, still, this list is quite long to be used for some complex visualisations without some adjustments.
- A year of publishing can be used to show/take into consideration temporal dynamics of any kind: it can be crucial since we are dealing with more than a decade.

Let's add the list of our categories into data_dict.

In [ ]:
data_dict['categories'] = categories
data_dict['categories']

## What's Next? (grouping the data to demonstrate interrelations of categories)

There are chances that we already have the design of our research (more or less) at the moment we open the csv-file.
<br>Maybe, we are relying on the first findings to tailor the possibilities of the particular dataset to our possible needs and, as a result, to formulate our questions.
<br><br>In both cases, we have two options.<br><br>
Firstly, we can explore our dataset in depth as a whole unit.
<br>Almost half a million channels, more than a decade of observations: all this can be insightful.

But, secondly, there's another option.<br>We can extract a more limited sample, say:
- the most active channels
- the most recent videos
- one particular thematic category (or categories)
- etc.

To ground our decisions better, let's take a look at the interrelations of the categorical data we're having: channel ids, video ids, years when videos were published and thematic categories of videos. 

### Unique Video Ids per Channel

In [ ]:
# Grouping the data.
channels_and_videos = data.groupby('channel_id')['video_id'].nunique()

# Let's make a horisontal barplot out of this!
(channels_and_videos
 .sort_values(ascending=False)
 .head(15)
.sort_values(ascending=True)
.plot(kind='barh', label=''))

# A few extra lines
median_video_per_channel = channels_and_videos.median().astype('int')
video_per_channel_95 = channels_and_videos.quantile(.95).astype('int')
video_per_channel_99 = channels_and_videos.quantile(.99).astype('int')

plt.axvline(x=median_video_per_channel, color='#e1a846', linestyle='dashdot',
            label=f'{median_video_per_channel}: median amount of videos per channel')
plt.axvline(x=video_per_channel_95, color='#2eac8b', linestyle='dashdot',
            label=f'{video_per_channel_95} videos per channel: 95%')
plt.axvline(x=video_per_channel_99, color='#d496ca', linestyle='dashdot',
            label=f'{video_per_channel_99} videos per channel: 99%')

plt.legend()

plt.title('15 most active channels\n(within our dataset)', fontsize=25)

plt.ylabel('channel id')
plt.xlabel('videos per channel')
plt.show()

**Channels and videos**
- There are a few leaders having more than a hundred videos.
- 99% of our channels show less commitment: they have no more than 5 videos per channel.
- This means, having more than **five** video uploaded is a **1%** minority affair.
- Moreover, about half of the channels settle for 2 videos or fewer than that.

### Unique Video Ids per Thematic Category

In [ ]:
videos_and_categories = (data
                         .groupby('video_category')
                         .agg(unique_video_ids=('video_id', 'nunique'))
                         .sort_values(ascending=False, by='unique_video_ids'))

(videos_and_categories
 .sort_values(ascending=True, by='unique_video_ids')
 .plot(kind='barh', legend=False))

plt.title('Thematic video categories\n',
          fontsize=25)
plt.ylabel('')
plt.xlabel('unique video ids')
plt.show()

In [ ]:
# Total amount of unique video ids.
unique_video_ids = data.video_id.nunique()
# Calculating share for each category.
videos_and_categories['%_of_all_video_ids'] = (round(videos_and_categories['unique_video_ids'] /
                                                    unique_video_ids,
                                                    4) * 100).astype('str')

videos_and_categories.sort_values(by='unique_video_ids', ascending=False).style.background_gradient()

**Unique video ids and thematic categories**

- The dominant category within our dataset is `Music` (almost 20% of unique video ids).
- Music videos are ahead by a wide margin from closest competitors: `Entertainment`, `Gaming`, `People and Blogs` and `Sports` (13-10%).
- `Nonprofits & Activism`, `Shows`, `Trailers` and `Movies` are the most underrepresented categories. Each of them comprises less than 0.7%. The latter two are almost non-existent within the dataset (a few hundredths of a percent per each).

### Age of the Channel

Now, let's dive into the time flow.

In [ ]:
# For each channel, we will find out what year first and last videos were published.
channels_and_years = (
    data.groupby('channel_id')
    .agg(first_year=('video_published_year', 'min'),
         last_year=('video_published_year', 'max'))
)
channels_and_years

In [ ]:
sns.heatmap(channels_and_years.sort_values(by='first_year').T)

plt.title('First and last years per channel\n(channels are sorted by the first year)\n', fontsize=25)
# Switching off the X-ticks to avoid the mess.
plt.xticks([], [])
plt.xlabel('channels')
plt.show()

In [ ]:
sns.heatmap(channels_and_years.sort_values(by='last_year').T)

plt.title('First and last years per channel\n(channels are sorted by the last year)\n', fontsize=25)
# Switching off the X-ticks.
plt.xticks([], [])
plt.xlabel('channels')
plt.show()

**Channels and years**
- For each channel, we have found out what year first and last videos were published.
<br>With 449&nbsp;980 unique channel ids, the resulting table seems to be impossible to fit to the screen.
- Thanks to our crazy idea to create a heatmap, even a tiny visualisation may suffice to see the pattern.
<br>For the majority of channels, the first year was also the last.<br>At the same time, we do have some 'long-livers', and the 'lighter' part is quite 'stripey'.

### Years and Video Categories

In [ ]:
(
    data.groupby('video_category')
    .agg(earliest_published=('video_published_year', 'min'),
         latestest_published=('video_published_year', 'max'))
).style.background_gradient(cmap='Blues_r')

In [ ]:
data_videos_and_years = (data
          .groupby(['video_category', 'video_published_year'], as_index=False)
          .agg(number_of_videos=('video_id', 'nunique'))
          .sort_values(by='number_of_videos', ascending=False))

sns.lineplot(data=data_videos_and_years,
             x="video_published_year", y='number_of_videos',
             hue="video_category")
plt.title('Videos of each category\npublished each year\n', fontsize=25)
plt.show()

Do you know what's horrible about the visualisation above?<br>18 categories in use are creating some fancy-looking rainbow that is more of an abstract painting, not a useful data visualisation.


Unfortunately, we can't just pick up a special palette: having more than five or seven categories is tricky not only in terms of colour choices but also for a human eye to perceive.


There are, however, different ways to solve this problem.

For instance, we can recreate our lineplot by using only the most active categories, say, five of them. We have already seen overall dynamics, so there's no need to focus on the rest.

In [ ]:
# We are reusing data grouping 'videos_and_categories' created previously:
# extracting top 5 video categories from it.
most_active_categories = videos_and_categories.head().index.to_list()

sns.lineplot(data=data_videos_and_years.query('video_category in @most_active_categories'),
             x="video_published_year", y='number_of_videos',
             hue="video_category",
             # Now, we are using a more inclusive palette.
             palette='colorblind')
plt.title('Videos of each category\npublished each year\n', fontsize=25)
plt.show()

That's better.

_____________

<br><br>
I hope this notebook was somehow helpful to you. Any recommendations, bits of advice and edits are more than welcome (showing gratitude in the form of upvoting would bring me joy, too).